# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 3:13PM,243501,10,8361570,Yusen – 3D Matrix,Released
1,Aug 4 2022 3:04PM,243500,20,8361056,"ACI Healthcare USA, Inc.",Released
2,Aug 4 2022 3:00PM,243482,10,0085924472,ISDIN Corporation,Released
3,Aug 4 2022 3:00PM,243482,10,0085924471,ISDIN Corporation,Released
4,Aug 4 2022 3:00PM,243482,10,0085924479,ISDIN Corporation,Released
5,Aug 4 2022 3:00PM,243482,10,0085924460,ISDIN Corporation,Released
6,Aug 4 2022 3:00PM,243482,10,0085924459,ISDIN Corporation,Released
7,Aug 4 2022 3:00PM,243482,10,0085924470,ISDIN Corporation,Released
8,Aug 4 2022 3:00PM,243482,10,0085924473,ISDIN Corporation,Released
9,Aug 4 2022 3:00PM,243482,10,0085924474,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,243495,Released,3
27,243496,Released,5
28,243497,Released,1
29,243500,Released,1
30,243501,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243495,NaN,3.0
243496,NaN,5.0
243497,NaN,1.0
243500,NaN,1.0
243501,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243436,0.0,1.0
243451,1.0,4.0
243455,0.0,1.0
243457,3.0,15.0
243461,1.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243436,0,1
243451,1,4
243455,0,1
243457,3,15
243461,1,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243436,0,1
1,243451,1,4
2,243455,0,1
3,243457,3,15
4,243461,1,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243436,,1
1,243451,1,4
2,243455,,1
3,243457,3,15
4,243461,1,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 3:13PM,243501,10,Yusen – 3D Matrix
1,Aug 4 2022 3:04PM,243500,20,"ACI Healthcare USA, Inc."
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation
25,Aug 4 2022 2:58PM,243497,12,Hush Hush
26,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc."
31,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC"
34,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC"
36,Aug 4 2022 2:46PM,243492,10,Emerginnova
37,Aug 4 2022 2:40PM,243491,16,"SHL Pharma, LLC"
38,Aug 4 2022 2:38PM,243490,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 4 2022 3:13PM,243501,10,Yusen – 3D Matrix,,1
1,Aug 4 2022 3:04PM,243500,20,"ACI Healthcare USA, Inc.",,1
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,,23
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,,1
4,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",,5
5,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC",,3
6,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC",,2
7,Aug 4 2022 2:46PM,243492,10,Emerginnova,,1
8,Aug 4 2022 2:40PM,243491,16,"SHL Pharma, LLC",,1
9,Aug 4 2022 2:38PM,243490,10,"Methapharm, Inc.",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:13PM,243501,10,Yusen – 3D Matrix,1,
1,Aug 4 2022 3:04PM,243500,20,"ACI Healthcare USA, Inc.",1,
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23,
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,1,
4,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5,
5,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC",3,
6,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC",2,
7,Aug 4 2022 2:46PM,243492,10,Emerginnova,1,
8,Aug 4 2022 2:40PM,243491,16,"SHL Pharma, LLC",1,
9,Aug 4 2022 2:38PM,243490,10,"Methapharm, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:13PM,243501,10,Yusen – 3D Matrix,1,
1,Aug 4 2022 3:04PM,243500,20,"ACI Healthcare USA, Inc.",1,
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23,
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,1,
4,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:13PM,243501,10,Yusen – 3D Matrix,1.0,NaN
1,Aug 4 2022 3:04PM,243500,20,"ACI Healthcare USA, Inc.",1.0,NaN
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23.0,NaN
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,1.0,NaN
4,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:13PM,243501,10,Yusen – 3D Matrix,1.0,0.0
1,Aug 4 2022 3:04PM,243500,20,"ACI Healthcare USA, Inc.",1.0,0.0
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23.0,0.0
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,1.0,0.0
4,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3408684,97.0,9.0
12,730434,4.0,0.0
15,730484,10.0,0.0
16,243491,1.0,0.0
19,243451,4.0,1.0
20,1217387,43.0,4.0
21,243483,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3408684,97.0,9.0
1,12,730434,4.0,0.0
2,15,730484,10.0,0.0
3,16,243491,1.0,0.0
4,19,243451,4.0,1.0
5,20,1217387,43.0,4.0
6,21,243483,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,97.0,9.0
1,12,4.0,0.0
2,15,10.0,0.0
3,16,1.0,0.0
4,19,4.0,1.0
5,20,43.0,4.0
6,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,97.0
1,12,Released,4.0
2,15,Released,10.0
3,16,Released,1.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,9.0,0.0,0.0,0.0,1.0,4.0,0.0
Released,97.0,4.0,10.0,1.0,4.0,43.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,9.0,0.0,0.0,0.0,1.0,4.0,0.0
1,Released,97.0,4.0,10.0,1.0,4.0,43.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,9.0,0.0,0.0,0.0,1.0,4.0,0.0
1,Released,97.0,4.0,10.0,1.0,4.0,43.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()